In [8]:
import yaml
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, XSD

RM = dict()

RM['ResearchInfrastructure'] = URIRef('http://www.oil-e.net/ontology/envri-rm.owl#ResearchInfrastructure')
RM['hasRepository'] = URIRef('http://www.oil-e.net/ontology/envri-rm.owl#hasRepository')

def process(document):
    n1 = URIRef(document['web'])

    g.add((n1, RDF.type, RM['ResearchInfrastructure']))
    g.add((n1, RDFS.label, Literal(document['label'])))
    
    if 'repositories' in document:
        for repository in document['repositories']:
            n2 = BNode()
            g.add((n1, RM['hasRepository'], n2))
            g.add((n2, RDFS.label, Literal(repository['name'])))
            g.add((n2, URIRef('http://example.org#url'), URIRef(repository['url'])))
    
    
g = Graph()

with open('data.yaml', "r") as f:
    for document in yaml.load_all(f, Loader=yaml.FullLoader):
        process(document)
    
g.serialize(destination='data.n3', format='n3')

In [11]:
import io
import pandas as pd
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

q = """
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT * WHERE {
?s a rm:ResearchInfrastructure .
}
"""

serializer = CSVResultSerializer(g.query(q))
output = io.BytesIO()
serializer.serialize(output)
display(pd.read_csv(io.StringIO(output.getvalue().decode('utf-8'))))

,s
0,http://emso.net
1,http://www.icos-ri.eu
2,http://www.lter.net
